In [1]:
import yaml
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine, text
import pandas as pd

In [2]:
def load_config(file_path="config.yaml"):
    with open(file_path, "r") as file:
        return yaml.safe_load(file)

In [3]:
config = load_config()
db_config = config["database"]

# Load credentials
db_user = db_config["user"]
db_password = db_config["password"]
db_host = db_config["host"]
db_port = db_config["port"]
db_name = db_config["name"]

# DB connection
conn = psycopg2.connect(
    dbname="postgres",
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)
conn.autocommit = True

In [4]:
db_name = "Candidatos"
try:
    with conn.cursor() as cur:
        cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
        print(f"Base de datos '{db_name}' creada exitosamente.")
except psycopg2.errors.DuplicateDatabase:
    print(f"La base de datos '{db_name}' ya existe.")
finally:
    conn.close()

La base de datos 'Candidatos' ya existe.


In [5]:

#Carga de datos desde .csv
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
df = pd.read_csv("candidates.csv", sep=';', encoding='latin-1')
df.to_sql('candidatos', engine, if_exists='replace', index=False, method="multi", chunksize=5000)

50000

In [6]:
# Leer datos de la tabla
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM candidatos LIMIT 1;"))
    rows = result.fetchall()

# Mostrar los resultados
print("Datos en 'candidatos':")
for row in rows:
    print(row)

Datos en 'candidatos':
('Bernadette', 'Langworth', 'leonard91@yahoo.com', '2021-02-26', 'Norway', 2, 'Intern', 'Data Engineer', 3, 3)


In [7]:

with engine.connect() as conn:
    df = pd.read_sql("SELECT * FROM candidatos LIMIT 5", conn)

In [8]:
df

,First Name,Last Name,Email,Application Date,Country,YOE,Seniority,Technology,Code Challenge Score,Technical Interview Score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7


In [9]:

df_transformed = df.copy()

df_transformed["hire"] = df_transformed.apply(
    lambda row: "HIRED" if (row["Code Challenge Score"] >= 7 and row["Technical Interview Score"] >= 7) else "DECLINE",
    axis=1
)
# Display transformed DataFrame
df_transformed.head()

,First Name,Last Name,Email,Application Date,Country,YOE,Seniority,Technology,Code Challenge Score,Technical Interview Score,hire
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3,DECLINE
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10,DECLINE
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9,HIRED
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1,DECLINE
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7,HIRED


In [ ]:

df_transformed.to_sql('tras_candidatos', engine, if_exists='replace', index=False, method="multi", chunksize=5000)

5

In [ ]:
# with engine.connect() as conn:
#     conn.execute(text("""
#         CREATE TABLE IF NOT EXISTS transformed_candidatos (

#         );
#     """))
#     conn.commit()  # Asegúrate de confirmar los cambios
#     print("Tabla 'transformed_candidatos' creada exitosamente en PostgreSQL.")

Tabla 'transformed_candidatos' creada exitosamente en PostgreSQL.


In [ ]:

# with engine.connect() as conn:
#     df_transformed.to_sql("transformed_candidatos", con=engine, if_exists="append", index=False)

# print("Transformed data stored successfully in 'transformed_candidatos'.")

Transformed data stored successfully in 'transformed_candidatos'.


In [11]:

with engine.connect() as conn:
    db_transformed_df = pd.read_sql("SELECT * FROM transformed_candidatos", conn)
    
db_transformed_df

,First Name,Last Name,Email,Application Date,Country,YOE,Seniority,Technology,Code Challenge Score,Technical Interview Score,hire
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3,DECLINE
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10,DECLINE
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9,HIRED
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1,DECLINE
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7,HIRED
5,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3,DECLINE
6,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10,DECLINE
7,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9,HIRED
8,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1,DECLINE
9,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7,HIRED
